# Результаты тестирования релиза 3.03.001.008

## Тест 1. Проверка исправления в рамках заявок 711900 и 710053 джоба pj005_F_SPU_STAJ_AGGR_p1Проверка корректности расчета стажа.

### Шаг 1
Выполнить запрос на подсчет стажа на DWH для снилса 36539384 

In [5]:
#В среде сопровождения DWH
staj_FAT = pd.read_csv('FAT_3.03.001.008_test1_step1.sql.csv')
stajdwh(staj_FAT)

Общий стаж:  12 лет 11 месяцев 21 дней 



SNILS
36539384    4671
Name: P_DAY1, dtype: int64

In [6]:
# В среде эксплуатации DWH
staj_RTP = pd.read_csv('RTP_3.03.001.008_test1_step1.sql.csv')
stajdwh(staj_RTP)

Общий стаж:  14 лет 8 месяцев 24 дней 



SNILS
36539384    5304
Name: P_DAY1, dtype: int64

### ШАГ 2
Выполнить запрос подсчет стажа на  DMR005 для снилса 36539384:

## Тест 2.	Проверка исправления в рамках заявки 727511 «Некорректные сведения о факте работы по адвокату на ВИР»

### Шаг 1
Выполнить в web-браузере (Google Chrome, Mozilla Firefox) запрос вида: 
    http://10.128.241.17:8080/report/query?query=SPUgetPeriodWorkingZLP&Snils=7117960360&PeriodStart=2019-11-01&PeriodEnd=2019-11-30

Ожидаемый результат: Для одного месяца для одного источника только 1 запись.

In [7]:
# Фактический результат
# Для одного месяца для одного источника только 1 запись:
pd.read_csv('SPUgetPeriodWorkingZLP_1.xml', encoding='utf-8', sep = '\n')

,"<?xml version=""1.0"" encoding=""UTF-8""?>"
0,"<vir:SPUgetPeriodWorkingZLResponse xmlns:vir=""..."
1,<vir:FamilyName>АЛДРСАЕНХ</vir:FamilyName>
2,<vir:FirstName>УБДЩДЕУР</vir:FirstName>
3,<vir:Patronymic>ЕТРУЛАДОТМ</vir:Patronymic>
4,<vir:Snils>7117960360</vir:Snils>
5,<vir:PeriodStart>2019-11-01</vir:PeriodStart>
6,<vir:PeriodEnd>2019-11-30</vir:PeriodEnd>
7,<vir:Periods>
8,<vir:Period>
9,<vir:RegNum>52049014129</vir:RegNum>


## Тест 3.	Проверка функции выборки данных для сервиса ВИР FBDPCharterNSU2 по РЗ -8045461.

### Шаг 1
выбор данных для проверки

Ожидаемый результат: 
<br>Результат: выборка по СНИЛС, подходящих для проверки

In [8]:
# Фактический результат в среде САИП:
# выборка по 1000 элементов
nsu = pd.read_csv('3.03.001.008_test3_step1.sql.csv')
nsu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747 entries, 0 to 1746
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   snils            1747 non-null   int64 
 1   formatted_snils  1747 non-null   object
 2   yes_nsu          1747 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 41.1+ KB


In [9]:
nsu.snils.nunique()

1739

### 8 дубляжей

### Шаг 2
НЕТ права на НСУ, не пенсионер 

In [11]:
# Проверяем снилсы у которых yes_nsu = 0
# Ожидаемый результат: 'Нет выплат по НСУ'
nsu_no = nsu[nsu['yes_nsu'] == 0]
for snils in nsu_no.snils:
    url = 'http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=' + str(snils)
    try:
        xml = objectify.parse(url).getroot()
        xml.Details
        if xml.Details == 'Нет выплат по НСУ':
            test_result = xml.Details, 'Тест пройден успешно'
        else:
            test_result = 'Error'
            print(snils, test_result)
    except:
        test_result = 'Error'
        print(snils, test_result, url)

12925631463 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=12925631463
9439642506 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=9439642506
894627174 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=894627174
1930732534 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=1930732534
131055964 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=131055964
769800869 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=769800869
5112236496 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=5112236496
2018527607 Error http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=2018527607


#### 8 снилсов у которых yes_nsu = 0 и yes_nsu = 1
#### Необходимо скорректировать скрипт отбора снилсов на шаге 1

### Шаг 3
ЕСТЬ право на НСУ, не пенсионер

In [12]:
# Проверяем снилсы у которых yes_nsu = 1
# Ожидаемый результат: xml в которой указана Наименование, например: "Ветераны боевых действий"
nsu_yes = nsu[nsu['yes_nsu'] == 1]
for snils in nsu_yes.snils:
    url = 'http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=' + str(snils)
    try:
        xml = objectify.parse(url).getroot()
        xml.Details
        if xml.Details == 'Нет выплат по НСУ':
            test_result = xml.Details, 'Error'
            print(snils, test_result, url)
        else:
            test_result = 'Тест прошел успешно'
#         print(snils, test_result, url)
    except:
        br_xml = pd.read_csv(url, encoding='utf-8', names = ['broken_xml'])
        text = br_xml.broken_xml[0]
        result = re.findall(r'Наименование>\w+', text)
        for name in result:
            name = name.replace("Наименование>", "")
            break
        test_result = name, 'Тест прошел успешно'
#         print(snils, test_result)

13892410879 ('Нет выплат по НСУ', 'Error') http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=13892410879
12556125342 ('Нет выплат по НСУ', 'Error') http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=12556125342
11008859728 ('Нет выплат по НСУ', 'Error') http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=11008859728
1837654972 ('Нет выплат по НСУ', 'Error') http://10.138.243.194:8080/report/query?query=FBDPCharterNSU2&Snils=1837654972


#### Необходимо проверить эти снилсы, почему yes_nsu = 1, и нет выплат. Возможно, нужно скорректировать скрипт отборы снилсов для проверки

In [1]:
import pandas as pd

In [2]:
def stajdwh(staj_dwh):
    staj = staj_dwh.P_DAY1.sum()
    years = staj // 360
    month = (staj - years*360) // 30
    days = staj - years*360 - month*30
    print('Общий стаж: ', years, 'лет', month, 'месяцев', days, 'дней \n',)
    return staj_dwh.groupby('SNILS').P_DAY1.sum().head()

In [3]:
from lxml import objectify

In [4]:
import re